## Setup

In [1]:

#
import pandas as pd
import numpy as np


# make sql database connection with pyodbc
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde_tabular;UID=sde;PWD=staff')


SyntaxError: invalid syntax (1196091081.py, line 5)

# Air Quality

In [38]:
pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/e3/23/00912e3236306ec52a06f95a08918cbb54f900419951726a20a8783a8507/geopandas-0.14.2-py3-none-any.whl.metadata
  Obtaining dependency information for fiona>=1.8.21 from https://files.pythonhosted.org/packages/1d/cf/31387b40d64f56631ad7eb3dfecd922ff17b2c3131a4765c7102743f4c0f/fiona-1.9.5-cp39-cp39-win_amd64.whl.metadata
     -------------------------------------- 51.1/51.1 kB 291.9 kB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/43/d0/cbe29a4dcf38ee7e72bf695d0d3f2bee21b4f22ee6cf579ad974de9edfc8/pyproj-3.6.1-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for shapely>=1.8.0 from https://files.pythonhosted.org/packages/51/18/7a6dfefc08d5899dfca0cde9ada7a58006d06cccc0818e68b2a60bdba8e3/shapely-2.0.2-cp39-cp39-win_amd64.whl.metadata
  Ob

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Noise

In [50]:
# Setup
import pandas as pd
import os
from arcgis import GIS
from arcgis.features import FeatureLayer
# Import geopandas
import geopandas as gpd
from shapely.geometry import Point

# Update SDE.Monitoring.SDE.Plan_Area_Noise
excel_file_path = r"F:\Research and Analysis\Noise\Monitoring\CNEL - Plan Areas\ALL_CNEL_PLANAREAS_RESULTS_(1991-2023).xlsx"

# Read the Excel file into a DataFrame
Noise23_df = pd.read_excel(excel_file_path)

# Keep only columns that you want
columns_to_keep = ['CNEL LAND USE', '1987 PAS#', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'PAS_NAME', 'LAND_USE', '2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']
Noise23_df = Noise23_df[columns_to_keep]

Noise23_df.columns = Noise23_df.columns.str.strip()

# PASNAME
Noise23_df['PAS_NAME'] = Noise23_df['PAS_NAME'].astype(str)

# Assuming df is your DataFrame
meltcolumns_to_keep = ['CNEL LAND USE', '1987 PAS#', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE']
id_columns = ['PAS_NAME']

# Specify the columns to melt (exclude the identifier column)
columns_to_melt = ['2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']

id_vars = id_columns + meltcolumns_to_keep

# Use pd.melt to reshape the DataFrame
Noise23_df = pd.melt(Noise23_df, id_vars=id_vars, value_vars=columns_to_melt, var_name='Year', value_name='CNEL_Maximum_Day_Value')

print(Noise23_df.head())
print(Noise23_df.columns)

# Extract only the year from the 'YEAR_OF_COUNT' column and drop "CNEL Maximum Day" from each value
Noise23_df['Year'] = Noise23_df['Year'].str.replace(' CNEL Maximum Day', '')



# Add 'Category' column
Noise23_df['Category'] = 'CNEL Maximum Day'

# Drop rows where 'CNEL_Maximum_Day_Value' is NaN
Noise23_df = Noise23_df.dropna(subset=['CNEL_Maximum_Day_Value'])

# Use lookup_dict to fill in LTINFO# Create a lookup dictionary to fill in LTINFO and ID
gis = GIS()


def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    feature_list = query_result.features
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data


# REST SERVICE data for lookup dictionary
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/6'

dfplannoisesde = get_fs_data(service_url)

dfplannoisesde.info()
columnstokeep = ['SITE_NAME', 'ID', 'LTINFO']
dfplannoisesde = dfplannoisesde.loc[:, columnstokeep]

# Drop duplicates based on 'SITE_NAME' and keep the first occurrence
unique_values = dfplannoisesde.drop_duplicates(subset='SITE_NAME', keep='first')

# Select specific columns for lookup
selected_columns = ['SITE_NAME', 'ID', 'LTINFO']

# Convert selected columns to dictionary
lookup_dict = unique_values[selected_columns].set_index('SITE_NAME').to_dict(orient='index')

# Display the dictionary
print(lookup_dict)

# Create new columns 'ID' and 'LTINFO' based on 'PAS_NAME' and look up dictionary
Noise23_df['ID'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('ID'))
Noise23_df['LTINFO'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('LTINFO'))

# Specify the columns to keep in Final_sdf
finalcolumns_to_keep = ['PAS_NAME', 'Year', 'CNEL_Maximum_Day_Value', 'Category','ID', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE', 'CNEL LAND USE', 'LTINFO', '1987 PAS#']

# Create a new DataFrame Final_df with the selected columns
Final_sdf = Noise23_df[finalcolumns_to_keep]

# Create a GeoDataFrame by converting 'LATITUDE' and 'LONGITUDE' to Point geometry
geometry = [Point(xy) for xy in zip(Final_sdf['MONITORING SITE LONGITUDE'], Final_sdf['MONITORING SITE LATITUDE'])]
Final_sdf = gpd.GeoDataFrame(Final_sdf, geometry=geometry)

# Field mapping so that columns match
field_mapping = {
    '1987 PAS#': 'PAS_1987',
    'ID': 'ID',
    'MONITORING SITE LATITUDE': 'LATITUDE',
    'MONITORING SITE LONGITUDE': 'LONGITUDE',
    'PAS_NAME': 'SITE_NAME',
    'LAND_USE': 'LAND_USE',
    'CNEL LAND USE': 'CNEL_LAND_USE',
    'Category': 'CATEGORY',
    'Year': 'YEAR_OF_COUNT',
    'CNEL_Maximum_Day_Value': 'COUNT_VALUE',
    'LTINFO': 'LTINFO'
}

# Create a new DataFrame Final_df with columns based on the field mapping
Final_sdf = Final_sdf.rename(columns=field_mapping)

# Display the updated Final_df
print(list(Final_sdf.columns))

# Save Final_df to a CSV file
Final_sdf.to_csv(r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Noise Drafts\NoiseThreshold23.csv', index=False)



               PAS_NAME             CNEL LAND USE 1987 PAS#  \
0            MOUNT ROSE       WILDERNESS/ROADLESS        30   
1         BIJOU MEADOWS  URBAN OUTDOOR RECREATION       101   
2              LAKEVIEW  HIGH DENSITY RESIDENTIAL        37   
3  ROUNDHILL COMMERCIAL                INDUSTRIAL        71   
4         MOUNTAIN VIEW   LOW DENSITY RESIDENTIAL       132   

   MONITORING SITE LATITUDE  MONITORING SITE LONGITUDE LAND_USE  \
0                 39.303580                -119.929140     CONS   
1                 38.921959                -119.961552      REC   
2                 39.248356                -119.971536      RES   
3                 38.984234                -119.941212   COM/PS   
4                 38.879719                -120.039483      RES   

                    Year  CNEL_Maximum_Day_Value  
0  2023 CNEL Maximum Day                     NaN  
1  2023 CNEL Maximum Day                     NaN  
2  2023 CNEL Maximum Day                     NaN  
3  2023 CNEL M

                PAS_NAME  Year  CNEL_Maximum_Day_Value          Category  \
37         FISH HATCHERY  2023                    54.2  CNEL Maximum Day   
39  KINGSBURY COMMERCIAL  2023                    63.3  CNEL Maximum Day   
44      INDUSTRIAL TRACT  2023                    55.6  CNEL Maximum Day   
54         TRUCKEE MARSH  2023                    48.3  CNEL Maximum Day   
60             ECHO VIEW  2023                    44.2  CNEL Maximum Day   

                                               LTINFO  \
37  {'ID': 179.0, 'LTINFO': 'https://monitoring.la...   
39  {'ID': 140.0, 'LTINFO': 'https://monitoring.la...   
44  {'ID': 81.0, 'LTINFO': 'https://monitoring.lak...   
54  {'ID': nan, 'LTINFO': 'https://monitoring.lake...   
60                                               None   

                                                   ID  
37  {'ID': 179.0, 'LTINFO': 'https://monitoring.la...  
39  {'ID': 140.0, 'LTINFO': 'https://monitoring.la...  
44  {'ID': 81.0, 'LTINFO': 'http

## Stream/CSCI

In [1]:
#Setup
#Create Dictionary Usring Rest Service data

import pandas as pd
import os
from arcgis import GIS
from arcgis.features import FeatureLayer
import numpy as np
import requests
import arcpy

gis = GIS()
# Connect to TRPA Enterprise GIS Portal *if it's a service only shared with org
# portal_user = "TRPA_PORTAL_ADMIN"
# portal_pwd = str(os.environ.get('Password'))
# portal_url = "https://maps.trpa.org/portal/"
wk_memory = "memory" + "\\"

#Set up Workspace
# set workspace and sde connections 
working_folder = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries"
workspace      = "C:\GIS\Scratch.gdb"
arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = r"C:\\GIS\\DB_CONNECT"


## CSV to be updated with new average
StreamAvg = os.path.join(working_folder,"ThresholdData_Stream.csv")


def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data

# Service URL
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8'

# Get Stream data as a Spatially Enabled DataFrame
sdfStreamHab = get_fs_data(service_url)

# Keep only necessary columns
columnstokeep = ['SITE_NAME', 'STATION_TYPE', 'DURATION', 'LATITUDE', 'LONGITUDE', 'LTINFO', 'COUNT_VALUE', 'YEAR_OF_COUNT']
sdfStreamHab = sdfStreamHab.loc[:, columnstokeep]

# Query only 2020-2022 data
sdfStreamHab['Date'] = pd.to_datetime(sdfStreamHab['YEAR_OF_COUNT'], format='%Y')
filtered_df = sdfStreamHab[
    (sdfStreamHab['Date'].dt.year.between(2020, 2022)) & 
    (sdfStreamHab['DURATION'] == 'Long-term')
]

# Calculate the average of the 'COUNT_VALUE' column
AvgCSCI = filtered_df['COUNT_VALUE'].mean()

# Update CSV ThresholdData_Stream.csv
csv_path = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv"
existing_df = pd.read_csv(csv_path)

Description = "Average CSCI score of all trend sites (48 sites)"
Year = '2020-2022'
Value = AvgCSCI

existing_entry = existing_df[
    (existing_df['Description'] == Description) &
    (existing_df['Year'] == Year)
]

if existing_entry.empty:
    new_entry = {'Description': Description, 'Year': Year, 'Value': Value}
    existing_df = existing_df.append(new_entry, ignore_index=True)

existing_df.to_csv(csv_path, index=False)
print(f"CSV file '{csv_path}' has been updated.")



CSV file 'F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv' has been updated.


C:\Users\snewsome\AppData\Local\Temp\ipykernel_22588\3748317425.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_df = existing_df.append(new_entry, ignore_index=True)


: 

In [ ]:
#Use F:Research and Analysis/Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv' to update SDE.tabular.....table

In [ ]:
#Calculate Rating or CSCI values and then find the percent of total streams in each rating

#Calculate Rating for CSCI value
#Define a function to categorize CSCI values based on ranges
def categorize_value(value):
    if 0 <= value < 0.6:
        return 'poor'
    elif 0.6 <= value < 0.8:
        return 'marginal'
    elif 0.8 <= value < 1.0:
        return 'good'
    else:
        return 'excellent'

#Find average for each rating- Poor, Marginal, Good, Excellent